In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import tools

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://ollama.com/library")

#start the loop here 
    #copy the text and open the url of this xpath in a new tab /html/body/div/main/div[2]/ul/li[1]/a/h2
        # wait 5 seconds 
            #click on this xpath /html/body/div/main/section[2]/div/div[1]/div/button  and wait 2 seconds
                # then click /html/body/div/main/section[2]/div/div[1]/div/div/div/div[2]/a
                    # then store all the text from this xpath /html/body/div/main/section[2]/div/div/div[2]    then this /html/body/div/main/section[2]/div/div/div[3]  /html/body/div/main/section[2]/div/div/div[68]  and so on untill you cant find any more 
                        #then make a json output for all the text you found as the first text from this xpath /html/body/div/main/div[2]/ul/li[1]/a/h2 as the main model and the texts found in the end as all the submodels
    # then move on to the second xpath on the main page /html/body/div/main/div[2]/ul/li[2]/a/h2
    #repeat this untill no more x paths are found 
    #tip you can use f"/html/body/div/main/div[2]/ul/li[{x}]/a/h2"



In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://ollama.com/library")

# Set up wait
wait = WebDriverWait(driver, 10)

# Store all models and their submodels
all_models = {}

# Iterate through each model on the main page
x = 1
while True:
    try:
        # Find each model's main link by changing the index
        model_xpath = f"/html/body/div/main/div[2]/ul/li[{x}]/a/h2"
        model_element = wait.until(EC.presence_of_element_located((By.XPATH, model_xpath)))
        
        # Get the model name
        model_name = model_element.text

        # Click to open the model page
        model_element.click()
        
        # Wait for the button to show up
        try:
            main_button_xpath = '/html/body/div/main/section[2]/div/div[1]/div/button'
            wait.until(EC.element_to_be_clickable((By.XPATH, main_button_xpath))).click()

            # Wait and click on the link to download all submodels
            submodels_link_xpath = '/html/body/div/main/section[2]/div/div[1]/div/div/div/div[2]/a'
            wait.until(EC.element_to_be_clickable((By.XPATH, submodels_link_xpath))).click()
        except Exception as e:
            print(f"Error: {e}")
            pass
        
        # Retrieve text from submodels sections
        submodels = []
        for submodel_idx in range(2, 69):  # Change 69 to the actual limit of submodels
            try:
                submodel_xpath = f'/html/body/div/main/section[2]/div/div/div[{submodel_idx}]'
                submodel_text = driver.find_element(By.XPATH, submodel_xpath).text
                submodels.append(submodel_text)
            except Exception:
                break
        
        # Store in the final dictionary
        all_models[model_name] = submodels

        # Go back to the main page
        driver.back()
        
        # Increment to the next main model
        x += 1
    except Exception:
        break

# Close the driver
driver.quit()

# Output the JSON
with open('models_data.json', 'w') as json_file:
    json.dump(all_models, json_file, indent=4)


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://ollama.com/library")

# Set up wait
wait = WebDriverWait(driver, 10)

# Store all models and their submodels
all_models = {}

# Save the initial window handle to switch back to later
main_tab = driver.current_window_handle

# Iterate through each model on the main page
x = 1
while True:
    try:
        # Find each model's main link by changing the index
        model_xpath = f"/html/body/div/main/div[2]/ul/li[{x}]/a/h2"
        model_element = wait.until(EC.presence_of_element_located((By.XPATH, model_xpath)))
        
        # Get the model name
        model_name = model_element.text
        
        # Get the model URL and open in a new tab
        model_url_xpath = f"/html/body/div/main/div[2]/ul/li[{x}]/a"
        model_url_element = wait.until(EC.element_to_be_clickable((By.XPATH, model_url_xpath)))
        model_url = model_url_element.get_attribute('href')
        
        driver.execute_script("window.open(arguments[0], '_blank');", model_url)
        
        # Switch to the new tab
        driver.switch_to.window(driver.window_handles[-1])
        
        # Perform the scraping actions
        try:
            # Click the button to reveal submodels
            main_button_xpath = '/html/body/div/main/section[2]/div/div[1]/div/button'
            wait.until(EC.element_to_be_clickable((By.XPATH, main_button_xpath))).click()

            # Click on the link to download all submodels
            submodels_link_xpath = '/html/body/div/main/section[2]/div/div[1]/div/div/div/div[2]/a'
            wait.until(EC.element_to_be_clickable((By.XPATH, submodels_link_xpath))).click()
        except Exception as e:
            print(f"Error: {e}")
            pass

        # Retrieve text from submodels sections
        submodels = []
        for submodel_idx in range(2, 69):  # Adjust 69 based on actual number of submodels
            try:
                submodel_xpath = f'/html/body/div/main/section[2]/div/div/div[{submodel_idx}]'
                submodel_text = driver.find_element(By.XPATH, submodel_xpath).text
                submodels.append(submodel_text)
            except Exception:
                break
        
        # Store in the final dictionary
        all_models[model_name] = submodels

        # Close the current tab and switch back to the main one
        driver.close()
        driver.switch_to.window(main_tab)

        # Increment to the next main model
        x += 1
    except Exception:
        break

# Close the driver
driver.quit()

# Output the JSON
with open('models_data.json', 'w') as json_file:
    json.dump(all_models, json_file, indent=4)
